In [1]:
import sys
import os
analysis_dir_path = '../'
sys.path.append(analysis_dir_path)

import models
import pandas as pd

In [2]:
filters = 32
kernels = 5
units = 128

In [3]:
CNN = f'CNN_{filters}_filters_{kernels}_kernels_predictions'
CNNA = f'CNN_Attention_{filters}_filters_{kernels}_kernels_predictions'
RNN = f'RNN_{units}_units_predictions'
LSTM = f'LSTM_{units}_layers_predictions'

In [4]:
MODEL = LSTM
COMMODITY = 'nickel_shift_20'
RULE_NUM = 20
THRESHOLD = 0.2
EXCLUDE = ['CNNA']
confidence_levels = [0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]

algo = ['correction', 'detection_correction']
ALGO = algo[1]

In [5]:
# df = models.npy_to_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'copper/test/predictions/test_{METRIC}_{ALGO}/', confidence_levels)
# df = models.npy_to_top_n_f1_bowpy(f'../{COMMODITY}/test/predictions/test/{MODEL}.csv', f'../{COMMODITY}/test/predictions/test_F1_{ALGO}', RULE_NUM)
df = models.npy_to_threshold_f1_bowpy(f'../{COMMODITY}/test/predictions/test/{MODEL}.csv', f'../{COMMODITY}/test/predictions/test_F1_{ALGO}', THRESHOLD, EXCLUDE)
df

f1 0.3673469387755102
Excluded model: CNNA_128_7, F1 Score: 0.3673469387755102
f1 0.28865979381443296
Excluded model: CNNA_64_3, F1 Score: 0.28865979381443296
f1 0.29268292682926833
f1 0.3537414965986394
f1 0.35971223021582743
f1 0.3795620437956204
Excluded model: CNNA_128_7, F1 Score: 0.3795620437956204
f1 0.2994011976047904
Excluded model: CNNA_64_3, F1 Score: 0.2994011976047904
f1 0.27999999999999997
f1 0.35971223021582743
f1 0.3565891472868217
f1 0.3846153846153846
Excluded model: CNNA_128_7, F1 Score: 0.3846153846153846
f1 0.32653061224489793
Excluded model: CNNA_64_3, F1 Score: 0.32653061224489793
f1 0.2978723404255319
f1 0.3410852713178295
f1 0.36974789915966383
f1 0.3577235772357723
Excluded model: CNNA_128_7, F1 Score: 0.3577235772357723
f1 0.34532374100719426
Excluded model: CNNA_64_3, F1 Score: 0.34532374100719426
f1 0.28571428571428575
f1 0.3247863247863248
f1 0.3418803418803419
f1 0.3548387096774194
Excluded model: CNNA_128_7, F1 Score: 0.3548387096774194
f1 0.340425531914

,pred,corr,true_positive,false_positive,rule0,rule1,rule2,rule3,rule4,rule5,...,rule24,rule25,rule26,rule27,rule28,rule29,rule30,rule31,rule32,rule33
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def calculate_metrics(df, rule_result_column):
    TP = ((df[rule_result_column] == 1) & (df['corr'] == 1)).sum()
    FP = ((df[rule_result_column] == 1) & (df['corr'] == 0)).sum()
    FN = ((df[rule_result_column] == 0) & (df['corr'] == 1)).sum()
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1, recall, precision

In [7]:
f1, recall, precision = calculate_metrics(df, "pred")


In [8]:
df.to_numpy().dump('data/test.npy')

In [9]:
import example
df = example.run_edcr()
df

quantity:0.061
NCi:[]
TP:4, FP:24, TN:345, FN:57
ep:0.001
[0.14285714285714285, 0.06557377049180328, 0.0898876404494382, 0, 0, 0, 0]
quantity:0.122
NCi:[]
TP:4, FP:24, TN:345, FN:57
ep:0.002
[0.14285714285714285, 0.06557377049180328, 0.0898876404494382, 0, 0, 0, 0]
quantity:0.183
NCi:[]
TP:4, FP:24, TN:345, FN:57
ep:0.003
[0.14285714285714285, 0.06557377049180328, 0.0898876404494382, 0, 0, 0, 0]
quantity:0.244
NCi:[]
TP:4, FP:24, TN:345, FN:57
ep:0.004
[0.14285714285714285, 0.06557377049180328, 0.0898876404494382, 0, 0, 0, 0]
quantity:0.305
NCi:[]
TP:4, FP:24, TN:345, FN:57
ep:0.005
[0.14285714285714285, 0.06557377049180328, 0.0898876404494382, 0, 0, 0, 0]
quantity:0.366
NCi:[]
TP:4, FP:24, TN:345, FN:57
ep:0.006
[0.14285714285714285, 0.06557377049180328, 0.0898876404494382, 0, 0, 0, 0]
quantity:0.427
NCi:[]
TP:4, FP:24, TN:345, FN:57
ep:0.007
[0.14285714285714285, 0.06557377049180328, 0.0898876404494382, 0, 0, 0, 0]
quantity:0.488
NCi:[]
TP:4, FP:24, TN:345, FN:57
ep:0.008
[0.14285714

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

ep:0.069
[0.0, 0.0, 0.0, 28, 0, 34, 0]
quantity:4.2700000000000005
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.07
[0.0, 0.0, 0.0, 28, 0, 34, 0]
quantity:4.331
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.07100000000000001
[0.0, 0.0, 0.0, 28, 0, 34, 0]
quantity:4.3919999999999995
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.07200000000000001
[0.0, 0.0, 0.0, 28, 0, 34, 0]
quantity:4.452999999999999
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.073
[0.0, 0.0, 0.0, 28, 0, 34, 0]
quantity:4.513999999999999
NCi:[4, 5

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

quantity:5.124
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.084
[0.0, 0.0, 0.0, 28, 0, 34, 0]
quantity:5.1850000000000005
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.085
[0.0, 0.0, 0.0, 28, 0, 34, 0]
quantity:5.246
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.08600000000000001
[0.0, 0.0, 0.0, 28, 0, 34, 0]
quantity:5.307
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.08700000000000001
[0.0, 0.0, 0.0, 28, 0, 34, 0]
quantity:5.367999999999999
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 2

...

ep:0.066
[0.0, 0.0, 0.0, 109, 81, 34, 12]
cci:[4, 7, 10, 35, 36, 13, 16, 31, 28, 25, 12, 18], new_pre:0.2385299217438372, pre:0.14285714285714285
quantity:4.087
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.067
[0.0, 0.0, 0.0, 109, 81, 34, 12]
cci:[4, 7, 10, 35, 36, 13, 16, 31, 28, 25, 12, 18], new_pre:0.2385299217438372, pre:0.14285714285714285
quantity:4.148
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.068
[0.0, 0.0, 0.0, 109, 81, 34, 12]
cci:[4, 7, 10, 35, 36, 13, 16, 31, 28, 25, 12, 18], new_pre:0.2385299217438372, pre:0.14285714285714285
quantity:4.2090000000000005
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.069
[0.0, 0.0, 0

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

ep:0.091
[0.0, 0.0, 0.0, 109, 81, 34, 12]
cci:[4, 7, 10, 35, 36, 13, 16, 31, 28, 25, 12, 18], new_pre:0.2385299217438372, pre:0.14285714285714285
quantity:5.611999999999999
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.092
[0.0, 0.0, 0.0, 109, 81, 34, 12]
cci:[4, 7, 10, 35, 36, 13, 16, 31, 28, 25, 12, 18], new_pre:0.2385299217438372, pre:0.14285714285714285
quantity:5.673
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.093
[0.0, 0.0, 0.0, 109, 81, 34, 12]
cci:[4, 7, 10, 35, 36, 13, 16, 31, 28, 25, 12, 18], new_pre:0.2385299217438372, pre:0.14285714285714285
quantity:5.734
NCi:[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
TP:0, FP:0, TN:369, FN:61
ep:0.094
[0.0, 0.0, 0.

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

,epsilon,pre,recall,F1,NSC,PSC,NRC,PRC
0,0.001,0.238532,0.42623,0.305882,0,81,0,12
1,0.002,0.238532,0.42623,0.305882,0,81,0,12
2,0.003,0.238532,0.42623,0.305882,0,81,0,12
3,0.004,0.238532,0.42623,0.305882,0,81,0,12
4,0.005,0.238532,0.42623,0.305882,0,81,0,12
...,...,...,...,...,...,...,...,...
94,0.095,0.000000,0.00000,0.000000,109,81,34,12
95,0.096,0.000000,0.00000,0.000000,109,81,34,12
96,0.097,0.000000,0.00000,0.000000,109,81,34,12
97,0.098,0.000000,0.00000,0.000000,109,81,34,12


In [10]:
print(f'Base model results\nf1: {f1}\nrecall: {recall}\nprecision: {precision}')

Base model results
f1: 0.0898876404494382
recall: 0.06557377049180328
precision: 0.14285714285714285
